In [93]:
import re
import codecs

log_pattern = r'(?P<datetime>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) - (?P<communicator>\w+) - (?P<level>\w+) - (?P<message>.*): \((?P<details>.*)\)'
def parse_log_file(file):
    cleaned_logs = []
    with open(file, 'r') as f:
        for line in f:
            match = re.match(log_pattern, line)
            if match:
                timestamp = match.group(1)
                communicator = match.group(2)
                log_level = match.group(3)
                message = match.group(4)
                details = match.group(5)

                # Filter out DEBUG messages
                if log_level != 'DEBUG':
                    cleaned_logs.append(
                        {'timestamp': timestamp, 'communicator': communicator, 
                         'level': log_level, 'message': message, 'details': details})
                    if (details[1:-1].split(',')[0]).startswith('\'counter') and (message == "valid msg"):
                        print(details)
                        print(message)
                        inner = details.split(":")[1].split(":")[0]  # "\\x00\\n\\xbc"
                        print(details.split(":"))
                        print(details.split(":")[1].split(":"))
                        print(inner.encode())
                        real_bytes = codecs.decode(inner.encode(), "unicode_escape")
                        print(real_bytes)
                        print(list(real_bytes.encode()))
                        l = list(real_bytes.encode())
                        counter = 0
                        for i in range(len(l)):
                            temp = len(l) - i - 1
                            counter += l[i] * (256 ** temp)
                        print(f"counter: {list(real_bytes.encode())} : {counter}")
            else:
                print("Log entry does not match the expected format:", line)
        return cleaned_logs


# Clean the raw logs
cleaned_logs = parse_log_file("../log/reciever/2025-11-07.log")

# Print cleaned logs
for log in cleaned_logs:
    print(log)


Log entry does not match the expected format: 2025-11-07 16:31:40,987 - reciever - DEBUG - debug message

Log entry does not match the expected format: 2025-11-07 16:31:40,987 - reciever - INFO - info message

Log entry does not match the expected format: 2025-11-07 16:31:40,987 - reciever - WARNING - warn message

Log entry does not match the expected format: 2025-11-07 16:31:40,987 - reciever - ERROR - error message

Log entry does not match the expected format: 2025-11-07 16:31:40,988 - reciever - CRITICAL - critical message

b'counter:\x00\x00\x02: hello', 18, -33, -33, 10, 0
valid msg
["b'counter", '\\x00\\x00\\x02', " hello', 18, -33, -33, 10, 0"]
['\\x00\\x00\\x02']
b'\\x00\\x00\\x02'
  
[0, 0, 2]
counter: [0, 0, 2] : 2
b'counter:\x00\x00\x07: hello', 18, -33, -33, 10, 0
valid msg
["b'counter", '\\x00\\x00\\x07', " hello', 18, -33, -33, 10, 0"]
['\\x00\\x00\\x07']
b'\\x00\\x00\\x07'
  
[0, 0, 7]
counter: [0, 0, 7] : 7
b'counter:\x00\x00\x0b: hello', 18, -34, -31, 10, 0
valid m